<a href="https://colab.research.google.com/github/ZeyadSabbah/TrivagoRecommenderSystem/blob/master/MoreEvaluatingModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating Models
## Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem


## Loading Libraries & Datasets

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
from datetime import datetime
import re
import random
import joblib
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [0]:
TrainMoreDataFilepath = './Datasets/clean_data/Sets/trainMore.csv'
valFilepath = './Datasets/clean_data/Sets/val.csv'
testFilepath = './Datasets/clean_data/Sets/test.csv'

TrainMoreData = pd.read_csv(TrainMoreDataFilepath)
valData = pd.read_csv(valFilepath)
testData = pd.read_csv(testFilepath)

## Validation & Test sets' Transformation & Scaling

In [0]:
#declaring features and label
features = TrainMoreData.drop(columns=['session_id', 'item_id', 'clickout']).columns.tolist()
label = ['clickout']

#dropping highly correlated features
FeaturesToDrop = ['NumberInImpressions', 'NumberInReferences', 'MeanPrice', 'MinPrice']
for feature in FeaturesToDrop:
  features.remove(feature)

X_train = TrainMoreData[features]
y_train = TrainMoreData[label]

### Preparation

In [0]:
valData_sessions_item = valData[['session_id', 'item_id', 'clickout']]
X_val = valData[features]
y_val = valData[label]

In [0]:
testData_sessions_item = testData[['session_id', 'item_id', 'clickout']]
X_test = testData[features]
y_test  = testData[label]

### Scaling

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])

from sklearn.compose import ColumnTransformer
full_pipeline = ColumnTransformer([
("num", num_pipeline, list(X_train))
])

# training set scaling
# X_train_scaled = full_pipeline.fit_transform(X_train)

# validation set scaling
X_val_scaled = full_pipeline.fit_transform(X_val)

# test set scaling
# X_test_scaled = full_pipeline.fit_transform(X_test)

## Evaluating Models

In [0]:
#function is from this repo https://gist.github.com/bwhite/3726239
def mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [0]:
def get_probabilities(model_path, X, session_item_dataset):
  '''
  Desc: function that gets the probability of each item being selected by the user, rerank the items in the session based on the probabilites

  Input: model_path: String with the name of the stored model
         X: array of scaled features of the dataset
         session_item_dataset: Pandas Dataframe with the sessions, items, and clickout
        
  Output: clickout_rank: List of lists that carries which item was selected in which rank
          RecommendationsDF: Pandas Dataframe to be transformed and merged to the Clickout Dataframe
  '''
  model = joblib.load(model_path)
  BothProbabilities = model.predict_proba(X)
  Probabilities = [Probability[1] for Probability in BothProbabilities]
  session_item_dataset['probability'] = Probabilities
  RecommendationsDF = session_item_dataset.groupby(['session_id'], sort=False).apply(lambda x: (x.sort_values('probability', ascending=False)))
  clickout_rank = RecommendationsDF.clickout
  clickout_rank = clickout_rank.reset_index().groupby('session_id').clickout.apply(list).values.tolist()
  return clickout_rank, RecommendationsDF

In [0]:
def ClassifReport(model_path, X, y):
  global y_pred
  model = joblib.load(model_path)
  y_pred = model.predict(X)
  return classification_report(y, y_pred)

In [0]:
def PrintMetrics(model_path, X, y, session_item_dataset):
  clickout_rank, RecommendationsDF = get_probabilities(model_path, X, session_item_dataset)
  MeanReciprocalRank = mean_reciprocal_rank(clickout_rank)
  print('Mean Reciprocal Rank : ', MeanReciprocalRank)
  print('=================================================')
  ClassificationReport = ClassifReport(model_path, X, y)
  print('Classification Report')
  print('=================================================')
  print(ClassificationReport)
  ConfMatrix = confusion_matrix(y_pred, y_val, labels=[1, 0])
  print('Confusion Matrix')
  print('================================================')
  print(ConfMatrix)
  return

### Without Resampling

#### Logistic Regression

In [0]:
PrintMetrics('./models/LR_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.4781245815526048
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.99      0.98   3249590
           1       0.49      0.12      0.20    149676

    accuracy                           0.96   3399266
   macro avg       0.73      0.56      0.59   3399266
weighted avg       0.94      0.96      0.94   3399266

Confusion Matrix
[[  18470   18856]
 [ 131206 3230734]]


In [0]:
PrintMetrics('./modelsMore/LR_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.47809684234265315
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.99      0.98   3249590
           1       0.49      0.12      0.19    149676

    accuracy                           0.96   3399266
   macro avg       0.73      0.56      0.59   3399266
weighted avg       0.94      0.96      0.94   3399266

Confusion Matrix
[[  18109   18731]
 [ 131567 3230859]]


Adding more data to train the model did not actually make a difference, almost the same.

#### Random Forest

In [0]:
#metrics without adding more data
PrintMetrics('./modelsMore/RF_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.561075864904377
Classification Report
               precision    recall  f1-score   support

           0       0.97      0.99      0.98   3249590
           1       0.59      0.31      0.40    149676

    accuracy                           0.96   3399266
   macro avg       0.78      0.65      0.69   3399266
weighted avg       0.95      0.96      0.95   3399266

Conusion Matrix
 [[3217334   32256]
 [ 103960   45716]]


In [14]:
#metrics after adding more data
PrintMetrics('./modelsMore/RF_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5676250317629644
Classification Report
              precision    recall  f1-score   support

           0       0.97      0.98      0.98   3249590
           1       0.50      0.35      0.41    149676

    accuracy                           0.96   3399266
   macro avg       0.73      0.67      0.70   3399266
weighted avg       0.95      0.96      0.95   3399266

Confusion Matrix
[[  53027   53373]
 [  96649 3196217]]


Mean Reciprocal Rank value increased slightly, which means training for more data helped the model enhance predictions and recommendations. The increase in the data would have been more significant if it was done on the whole dataset.

####XGBoost

In [0]:
PrintMetrics('./models/XGBoost_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5655116470914219
Classification Report
               precision    recall  f1-score   support

           0       0.97      0.99      0.98   3249590
           1       0.57      0.30      0.40    149676

    accuracy                           0.96   3399266
   macro avg       0.77      0.65      0.69   3399266
weighted avg       0.95      0.96      0.95   3399266

Conusion Matrix
 [[3215292   34298]
 [ 104293   45383]]


Train a model with the least parameters and see what the output is.

In [0]:
PrintMetrics('./models/XGBoost_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5595317920589848
Classification Report
               precision    recall  f1-score   support

           0       0.96      1.00      0.98   3249590
           1       0.70      0.11      0.19    149676

    accuracy                           0.96   3399266
   macro avg       0.83      0.55      0.58   3399266
weighted avg       0.95      0.96      0.94   3399266

Conusion Matrix
 [[3242537    7053]
 [ 133206   16470]]


In [0]:
PrintMetrics('./models/XGBoost_model.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5837948821477086 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report
               precision    recall  f1-score   support

           0       0.96      1.00      0.98   3249590
           1       0.00      0.00      0.00    149676

    accuracy                           0.96   3399266
   macro avg       0.48      0.50      0.49   3399266
weighted avg       0.91      0.96      0.93   3399266

Confusion Matrix
 [[3249590       0]
 [ 149676       0]]


In [0]:
PrintMetrics('./modelsMore/XGBoost_model.pkl', X_val_scaled, y_val, valData_sessions_item)

### With SMOTE

####Logistic Regression

In [0]:
PrintMetrics('./modelsSMOTE/LR_modelSMOTE.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.4694871489433295
Classification Report
               precision    recall  f1-score   support

           0       0.98      0.80      0.88   3249590
           1       0.13      0.64      0.21    149676

    accuracy                           0.79   3399266
   macro avg       0.55      0.72      0.55   3399266
weighted avg       0.94      0.79      0.85   3399266

Conusion Matrix
 [[2594331  655259]
 [  53261   96415]]


#### Random Forest

In [0]:
PrintMetrics('./modelsSMOTE/RF_SMOTEmodel.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5540867336740866
Classification Report
               precision    recall  f1-score   support

           0       0.97      0.99      0.98   3249590
           1       0.58      0.31      0.40    149676

    accuracy                           0.96   3399266
   macro avg       0.78      0.65      0.69   3399266
weighted avg       0.95      0.96      0.95   3399266

Conusion Matrix
 [[3217129   32461]
 [ 103994   45682]]


In [0]:
PrintMetrics('./modelsSMOTE/RF_modelSMOTE.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5156955682610294
Classification Report
               precision    recall  f1-score   support

           0       0.98      0.64      0.78   3249590
           1       0.08      0.71      0.15    149676

    accuracy                           0.64   3399266
   macro avg       0.53      0.67      0.46   3399266
weighted avg       0.94      0.64      0.75   3399266

Conusion Matrix
 [[2084144 1165446]
 [  43730  105946]]


#### XGBoost

In [0]:
PrintMetrics('./modelsSMOTE/XGBoostSMOTE.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5312985317511649
Classification Report
               precision    recall  f1-score   support

           0       0.98      0.70      0.82   3249590
           1       0.10      0.70      0.17    149676

    accuracy                           0.70   3399266
   macro avg       0.54      0.70      0.49   3399266
weighted avg       0.94      0.70      0.79   3399266

Conusion Matrix
 [[2272863  976727]
 [  44829  104847]]


### With Undersampling

#### Logistic Regression

In [0]:
PrintMetrics('./modelsUnderSampling/LR_modelUndersampling.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.46915470104643736
Classification Report
               precision    recall  f1-score   support

           0       0.98      0.80      0.88   3249590
           1       0.13      0.64      0.21    149676

    accuracy                           0.79   3399266
   macro avg       0.55      0.72      0.55   3399266
weighted avg       0.94      0.79      0.85   3399266

Conusion Matrix
 [[2596866  652724]
 [  53332   96344]]


#### Random Forest

In [0]:
PrintMetrics('./modelsUnderSampling/RF_modelUnderSampling.pkl', X_val_scaled, y_val, valData_sessions_item)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Mean Reciprocal Rank :  0.5551711886318428
Classification Report
               precision    recall  f1-score   support

           0       0.98      0.88      0.93   3249590
           1       0.19      0.61      0.29    149676

    accuracy                           0.87   3399266
   macro avg       0.59      0.75      0.61   3399266
weighted avg       0.95      0.87      0.90   3399266

Conusion Matrix
 [[2863288  386302]
 [  58368   91308]]


In [0]:
clickout_rank, DF_Probabilities = get_probabilities('./modelsUnderSampling/RF_modelUnderSampling.pkl',
                                                    X_val_scaled, valData_sessions_item)
mean_reciprocal_rank(clickout_rank)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0.5551711886318428

#### XGBoost

In [0]:
PrintMetrics('./modelsUnderSampling/XGBoost_UnderSampling.pkl', X_val_scaled, y_val, valData_sessions_item)

In [0]:
clickout_rank, DF_Probabilities = get_probabilities('./modelsUnderSampling/XGBoost_UndersSampling.pkl', X_val_scaled, valData_sessions_item)
mean_reciprocal_rank(clickout_rank)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0.5601924601401805

## Submission

In the future, the challenge might be a public competition, in that case transformation of the output is needed.

In [0]:
test_set_filepath = './Datasets/raw_data/test.csv'
test_set = pd.read_csv(test_set_filepath)
test_set.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [0]:
submission_format_filepath = './Datasets/raw_data/submission_popular.csv'
submission_format = pd.read_csv(submission_format_filepath)
submission_format.head()

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 222702...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 222702...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 ...


In [0]:
def transform_Recommendations(clickout_dataframe, RecommendationsDF):
  ListOfItems = RecommendationsDF.reset_index(drop=True)[['session_id', 'item_id']].groupby('session_id', sort=False).item_id.apply(pd.Series.tolist).tolist()
  SessionsListOfItems = pd.DataFrame({'session_id':RecommendationsDF.session_id.unique().tolist(),
                                      'item_recommendations':ListOfItems})
  SessionsListOfItems.item_recommendations = SessionsListOfItems.item_recommendations.apply(lambda x: ' '.join(x))
  data = clickout_dataframe.merge(SessionsListOfItems, on='session_id', how='left')
  return data

In [0]:
def get_probabilities_submission(model_name, X, session_item_dataset):
  '''
  Desc: function that gets the probability of each item being selected by the user, rerank the items in the session based on the probabilites

  Input: model_path: String with the name of the stored model
         X: array of scaled features of the dataset
         session_item_dataset: Pandas Dataframe with the sessions, items, and clickout
        
  Output: clickout_rank: List of lists that carries which item was selected in which rank
          RecommendationsDF: Pandas Dataframe to be transformed and merged to the Clickout Dataframe
  '''
  model = joblib.load(model_name)
  BothProbabilities = model.predict_proba(X)
  Probabilities = [Probability[1] for Probability in BothProbabilities]
  session_item_dataset['probability'] = Probabilities
  RecommendationsDF = session_item_dataset.groupby(['session_id'], sort=False).apply(lambda x: (x.sort_values('probability', ascending=False)))
  return RecommendationsDF

In [0]:
ListOfItems = Output.reset_index(drop=True)[['session_id', 'item_id']].groupby('session_id', sort=False).item_id.apply(pd.Series.tolist).tolist()
SessionsListOfItems = pd.DataFrame({'session_id':Output.session_id.unique().tolist(),
                                    'item_recommendations':ListOfItems})
SessionsListOfItems.item_recommendations = SessionsListOfItems.item_recommendations.apply(lambda x: ' '.join(x))
SessionsListOfItems.head()

In [0]:
from data_transformation import data_transformation

test_clickout = test_set[test_set.action_type=='clickout item'].groupby('session_id').tail(1)
test_clickout = test_clickout[['user_id', 'session_id', 'timestamp', 'step']]
test_set_transformed = data_transformation.transform_data(test_set)
test_session_item = test_set_transformed[['session_id', 'item_id']]
X_test_submission = test_set_transformed[features]
X_test_submission_scaled = full_pipeline.fit_transform(X_test_submission)
RecommendationsDF = get_probabilities_submission('modelsUnderSampling/LR_modelUndersampling.pkl', X_test_submission_scaled, test_session_item)
SubmissionDF = transform_Recommendations(test_clickout, RecommendationsDF)
SubmissionDF.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,user_id,session_id,timestamp,step,item_recommendations
0,004A07DM0IDW,1d688ec168932,1541555799,7,399441 2033381 127131 2059240 1724779 103357 6...
1,009RGHI3G9A3,f05ab0de907e2,1541570940,2,10884872 7065316
2,00Y1Z24X8084,26b6d294d66e7,1541651823,2,3853058 4476010 7101352 3833012 2714480 934300...
3,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,7950162 4115018 3180004 2817590 6434434 356572...
4,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,7304020 559056 693596 1451247 1177554 1963879 ...


In [0]:
SubmissionDF.to_csv('./Datasets/clean_data/RecommendationsSubmission.csv', index=False)